In [ ]:
#graph average precision at k with k ranging from 1 to 20

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

#using trained moddel generate top k recommendations for each user and then calculate average precision at k
#build a dictionary of k values and their corresponding average precision at k

k_values = {}

#generate top 20 recommendations for each user
recommendations = {}

k = 20

for uid, uvec in user_profiles_norm.items():
    # movies user has already rated (train)
    seen = set(train_ratings[train_ratings['userId'] == uid]['movieId'])
    
    # candidate pool = all movies minus seen
    candidates = genre_matrix.drop(index=seen, errors='ignore')
    
    # compute cosine similarity
    sims = cosine_similarity(uvec.reshape(1, -1), candidates.values)[0]
    
    # grab the Top‑K movieIds
    top_k_ids = candidates.index[np.argsort(sims)[-K:][::-1]]
    recommendations[uid] = list(top_k_ids)

#calculate the average precision at k ranging from 1 to 20 and use to fill k_values dictionary
k = 1
for k in range(1, 21):

    precision_at_k = {}
    for uid, recs in recommendations.items():
        a = 0
        for movie in recs[:k]:
            if movie in good_test_ratings['movieId'][good_test_ratings['userId'] == uid].values:
                a += 1
        precision_at_k[uid] = a / k

        #find average precision at k for all users
    sum_precision = 0
    for uid, recs in precision_at_k.items():
        sum_precision += recs

    average_precision = sum_precision / len(recommendations)
    k_values[k] = average_precision


#plot k_values
plt.plot(list(k_values.keys()), list(k_values.values()))
plt.xlabel('K (number of recommendations)')
plt.ylabel('Average Precision at K')
plt.title('Average Precision at K vs K')
plt.xticks(list(k_values.keys()))
plt.show()